# CatBoost - v2

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/sample_submission.csv
/kaggle/input/train_v2.csv
/kaggle/input/test_v2.csv


In [2]:
train = pd.read_csv('/kaggle/input/train_v2.csv', index_col= None)
test = pd.read_csv('/kaggle/input/test_v2.csv', index_col= None)
submission = pd.read_csv('/kaggle/input/sample_submission.csv', index_col= 'ID')

## 1. 데이터 나누기

In [3]:
X = train.drop('임신 성공 여부', axis=1)
X_test = test.copy()

y = train['임신 성공 여부'].values

## 2. 최적화된 파라미터

In [4]:
max_params = {
    'bagging_temperature': 0.6322778060523135,
    'border_count': 243,
    'depth': 8,
    'l2_leaf_reg': 9.136867897737297,
    'learning_rate': 0.05093949002181776,
    'loss_function': 'Logloss',
    'eval_metric': 'AUC',
    'iterations': 2500,
    'random_seed': 1991,
    'verbose': 200
}

## 3. 모델 훈련

In [5]:
from sklearn.model_selection import StratifiedKFold

# 층화 K 폴드 교차 검증기
folds = StratifiedKFold(n_splits=10, shuffle=True, random_state=1991)

In [6]:
# from sklearn.metrics import roc_auc_score
# from catboost import CatBoostClassifier, Pool

# # OOF 방식으로 모델 훈련, 검증, 예측
# oof_val_preds_catboost = np.zeros(X.shape[0])
# oof_test_preds_catboost = np.zeros(X_test.shape[0])

# for idx, (train_idx, valid_idx) in enumerate(folds.split(X, y)):
#     print('#'*40, f'폴드 {idx+1} / 폴드 {folds.n_splits}', '#'*40)
    
#     X_train, y_train = X[train_idx], y[train_idx]
#     X_valid, y_valid = X[valid_idx], y[valid_idx]
    
#     dtrain = Pool(X_train, y_train, cat_features=None)
#     dvalid = Pool(X_valid, y_valid, cat_features=None)
    
#     cat_model = CatBoostClassifier(**max_params)
#     cat_model.fit(dtrain, 
#                   eval_set=dvalid, 
#                   early_stopping_rounds=200, 
#                   verbose=500)
    
#     # 테스트 데이터 예측
#     oof_test_preds_catboost += cat_model.predict_proba(X_test)[:, 1] / folds.n_splits
#     # 검증 데이터 예측
#     oof_val_preds_catboost[valid_idx] += cat_model.predict_proba(X_valid)[:, 1]
    
#     # 검증 데이터 ROC-AUC 계산
#     roc_auc = roc_auc_score(y_valid, oof_val_preds_catboost[valid_idx])
#     print(f'폴드 {idx+1} ROC-AUC : {roc_auc}\n')

In [7]:
from sklearn.metrics import roc_auc_score
from catboost import CatBoostClassifier, Pool

# OOF 방식으로 모델 훈련, 검증, 예측
oof_val_preds_catboost = np.zeros(X.shape[0])
oof_test_preds_catboost = np.zeros(X_test.shape[0])

for idx, (train_idx, valid_idx) in enumerate(folds.split(X, y)):
    print('#'*40, f'폴드 {idx+1} / 폴드 {folds.n_splits}', '#'*40)

    # ✅ DataFrame 인덱싱 수정 (.iloc 사용)
    X_train, y_train = X.iloc[train_idx], y[train_idx]
    X_valid, y_valid = X.iloc[valid_idx], y[valid_idx]

    # ✅ CatBoost 범주형 컬럼 추가
    dtrain = Pool(X_train, y_train, cat_features=None)
    dvalid = Pool(X_valid, y_valid, cat_features=None)

    # 모델 학습
    cat_model = CatBoostClassifier(**max_params)
    cat_model.fit(dtrain, 
                  eval_set=dvalid, 
                  early_stopping_rounds=200, 
                  verbose=500)
    
    # ✅ 테스트 데이터도 Pool 변환
    dtest = Pool(X_test, cat_features=None)

    # ✅ 테스트 데이터 예측
    oof_test_preds_catboost += cat_model.predict_proba(dtest)[:, 1] / folds.n_splits
    # ✅ 검증 데이터 예측
    oof_val_preds_catboost[valid_idx] = cat_model.predict_proba(dvalid)[:, 1]

    # 검증 데이터 ROC-AUC 계산
    roc_auc = roc_auc_score(y_valid, oof_val_preds_catboost[valid_idx])
    print(f'폴드 {idx+1} ROC-AUC : {roc_auc}\n')

######################################## 폴드 1 / 폴드 10 ########################################
0:	test: 0.7076247	best: 0.7076247 (0)	total: 115ms	remaining: 4m 46s
500:	test: 0.7399775	best: 0.7400378 (473)	total: 24.6s	remaining: 1m 37s
Stopped by overfitting detector  (200 iterations wait)

bestTest = 0.7400377988
bestIteration = 473

Shrink model to first 474 iterations.
폴드 1 ROC-AUC : 0.7400377987838436

######################################## 폴드 2 / 폴드 10 ########################################
0:	test: 0.7164902	best: 0.7164902 (0)	total: 53ms	remaining: 2m 12s
500:	test: 0.7390021	best: 0.7391402 (418)	total: 24.5s	remaining: 1m 37s
Stopped by overfitting detector  (200 iterations wait)

bestTest = 0.7391402254
bestIteration = 418

Shrink model to first 419 iterations.
폴드 2 ROC-AUC : 0.7391402253617091

######################################## 폴드 3 / 폴드 10 ########################################
0:	test: 0.7041943	best: 0.7041943 (0)	total: 51ms	remaining: 2m 7s
500:	test: 0

## 4. 최종 결과

In [11]:
print('최종 OOF 검증 데이터 ROC-AUC:', roc_auc_score(y, oof_val_preds_catboost))

최종 OOF 검증 데이터 ROC-AUC: 0.7399599375435523


In [12]:
# OOF 검증 예측 결과 저장
oof_val_preds_df = pd.DataFrame({'ID': train.index, 'oof_val_preds': oof_val_preds_catboost})
oof_val_preds_df.to_csv("cat_v2_oof_val_predictions.csv", index=False)

# OOF 테스트 예측 결과 저장
oof_test_preds_df = pd.DataFrame({'ID': test.index, 'oof_test_preds': oof_test_preds_catboost})
oof_test_preds_df.to_csv("cat_v2_oof_test_predictions.csv", index=False)

In [13]:
submission['probability'] = oof_test_preds_catboost
submission.to_csv('cat_v2_submission.csv')